# Use Intent Classification to orchestrate multiple sub-agents 



In the hierarchical model of multi-agent collaboration, an alternative approach to using a supervisor agent is employing a Large Language Model (LLM) for intent classification and request routing. This method leverages the LLM's natural language understanding capabilities to analyze user inputs and determine which specialized agent should handle the request. For instance, in a mortgage services scenario, the LLM can distinguish between payment processing and loan application inquiries, directing them to the appropriate agent. This approach streamlines the interaction flow, reducing the need for complex decision-making hierarchies and allowing for more flexible and scalable system architecture.

In this notebook, you will see how we can use the LLM intent classification to route requests to the two previous agents you built in [01_create_agents_and_kbs.ipynb](./01_create_agents_and_kbs.ipynb). 



First step is to install the pre-requisites packages. NOTE: You only need to do this is this is the first notebook you are running. 

In [59]:
# !pip install --upgrade -q -r requirements.txt
# !pip install --upgrade -q boto3 botocore 

In [1]:
import time
import boto3
import logging
import pprint
import json

%load_ext autoreload
%autoreload 2

from agent import AgentsForAmazonBedrock

In [2]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
bedrock_client = boto3.client('bedrock-runtime')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

suffix = f"{region}-{account_id}"
bucket_name = f'mac-workshop-{suffix}'
agent_foundation_models = [
    "anthropic.claude-3-haiku-20240307-v1:0",
    "anthropic.claude-3-sonnet-20240229-v1:0", 
    ]

In [4]:
agents = AgentsForAmazonBedrock()

In [5]:
sub_agent_names = ["existing_mortgage_agent", "mortgage_application_agent"]

In [6]:
def classifyIntentByLLM(input_text, bedrock_runtime, model_id, 
                        top_p=1, temp=0, system=''):  
    messages = [
        {
        "role": 'user',
        "content": [ {"type": "text", "text":
f"""
You will be given some user input. Classify that input according to which bot would be
the most appropriate one to respond that input.

EXISTING: can retrieve the latest information about an existing mortgage, like principal balance and
payment information.

APPLICATION: handle requests related to mortgage applications that are in process. creating, managing, 
and completing these mortgage applications. help find status of required application documentation.
assumes that the mortgage type is already determined. unable to handle general questions about mortgages
like tradeoffs between different mortgage types.

MORTGAGE_TYPE_KB: handle general mortgage questions about different mortgage types like comparing 15-year vs 30-year

OTHER: not applicable for any of the bots available.

Here is the user input text:
<input_text>
{input_text}
</input_text>

Return only the name of the bot, with no preamble.
"""
        }]
        }
    ]  
    body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 25,
            "messages": messages,
            "temperature": temp,
            "top_p": top_p,
            "system": system,
            "stop_sequences":["assistant"]
        }  
    )  
    
    response = bedrock_runtime.invoke_model(body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())
    # print(response_body)
    
    return response_body['content'][0]['text']

In [ ]:
classifyIntentByLLM("what is the status of my application?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

In [ ]:
classifyIntentByLLM("what is better, 30 year mortgage or 15 year?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

In [ ]:
classifyIntentByLLM("why would I pick a 15 year mortgage over a 30 year?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

In [ ]:
classifyIntentByLLM("what is the status of the documents I need to provide?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

In [ ]:
classifyIntentByLLM("when is my next payment due?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

In [ ]:
classifyIntentByLLM("I am customer 288. What is my existing balance?", bedrock_client, 
                model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

In [ ]:
INTENT_TO_ID_MAP = {
    "EXISTING": agents.get_agent_id_by_name("existing_mortgage_agent"),
    "APPLICATION": agents.get_agent_id_by_name("mortgage_application_agent")
}
INTENT_TO_ID_MAP 

In [ ]:
from knowledge_base import BedrockKnowledgeBaseHelper
helper = BedrockKnowledgeBaseHelper()
kb_id = helper.get_kb()
kb_arn = f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"

print(kb_id)
print(kb_arn)

In [19]:
def invokeByIntent(input_text, session_id, session_state={}):
    input_intent = classifyIntentByLLM(input_text, bedrock_client, 
                            model_id = "anthropic.claude-3-haiku-20240307-v1:0", temp=0.5, top_p=0.9)

    if input_intent == "EXISTING":
        # print('invoking Existing Mortgage agent...')
        response = agents.invoke(input_text, INTENT_TO_ID_MAP["EXISTING"], 
                                              session_id=session_id, session_state=session_state)
    elif input_intent == "APPLICATION":
        # print('invoking Mortgage Application agent...')
        response = agents.invoke(input_text, INTENT_TO_ID_MAP["APPLICATION"], 
                                              session_id=session_id, session_state=session_state)
    elif input_intent == "MORTGAGE_TYPE_KB":
        rag_response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                    "text": input_text
                },
            retrieveAndGenerateConfiguration={
                    "type": "KNOWLEDGE_BASE",
                    "knowledgeBaseConfiguration": {
                        'knowledgeBaseId': kb_id,
                        "modelArn": f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_models[0]}",
                        "retrievalConfiguration": {
                            "vectorSearchConfiguration": {
                                "numberOfResults":5
                            } 
                        }
                    }
                }
            )
        response = rag_response['output']['text']
    else:
        response = "Sorry, I don't have that expertise. Please ask a different question."

    return response

In [15]:
sup_session_id = '999'

In [ ]:
%%time
print(invokeByIntent("my id is 8953. when is my payment due?", sup_session_id))

In [ ]:
%%time
invokeByIntent("compare and contrast 15-year vs 30-year mortgage type", sup_session_id)

In [ ]:
%%time
invokeByIntent("I am customer 948. what documents do i still need to provide for my application?", 
                sup_session_id)

In [ ]:
%%time
invokeByIntent("what documents have i already provided for my application?", sup_session_id)

In [ ]:
%%time
invokeByIntent("my ID is 9856. what is my principal balance on my existing mortgage?", 
            sup_session_id)

In [ ]:
%%time 
invokeByIntent("my customer id is 8953. when is my payment due?", sup_session_id)

In [ ]:
%%time 
invokeByIntent("what considerations are there when deciding on a 30 year mortage?", sup_session_id)

##### Invoke Agent by Intent, with prompt attribute

In [ ]:
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')

session_state = {
    "promptSessionAttributes": {
        "customer_ID": "498",
        "today": today
    }
}
session_state

In [ ]:
%%time
sup_session_id = "111"
print(invokeByIntent("what docs do I still owe you?", 
                    sup_session_id, session_state=session_state))

In [ ]:
%%time
sup_session_id = "222"
print(invokeByIntent("how many years until I reach the mortgage maturity date?", 
                        sup_session_id, session_state=session_state))

### Quick performance test

In [32]:
import uuid 
import time
import numpy as np

def query_loop_by_intent(query, num_invokes):
    latencies = []
    for i in range(num_invokes):
        _session_id = str(uuid.uuid1())
        _start_time = time.time()
        resp = invokeByIntent(query, _session_id)
        _end_time = time.time()
        latencies.append(_end_time - _start_time)

    print(f'\n\nInvoked by intent {num_invokes} times.')
    # get sum of total latencies
    total_time = sum(latencies)
    # get average latency
    avg_time = total_time / num_invokes
    # get p90 latency
    p90_time = np.percentile(latencies, 90)

    print(f'Average latency: {avg_time:.1f}, P90 latency: {p90_time:.1f}')

In [ ]:
query_loop_by_intent("I am customer 999. how many years until the mortgage maturity date?", 25)

In [34]:
import uuid 
import time
import numpy as np

def classification_loop(query, num_invokes, runtime, model):
    latencies = []
    for i in range(num_invokes):
        _session_id = str(uuid.uuid1())
        _start_time = time.time()
        resp = classifyIntentByLLM(query, runtime, model)
        _end_time = time.time()
        latencies.append(_end_time - _start_time)

    print(f'\n\nClassified by intent {num_invokes} times.')
    # get sum of total latencies
    total_time = sum(latencies)
    # get average latency
    avg_time = total_time / num_invokes
    # get p90 latency
    p90_time = np.percentile(latencies, 90)

    print(f'Average latency: {avg_time:.1f}, P90 latency: {p90_time:.1f}')

In [ ]:
classification_loop("I'm customer 99. what's my balance?", 25, 
                    bedrock_client, agent_foundation_models[0])